<a href="https://colab.research.google.com/github/alyaa21/Apple-Detection/blob/master/convert_data_to_yolo_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
import os 

In [24]:
os.chdir('/content/drive/MyDrive/OID/csv_folder')
classes_data= pd.read_csv('class-descriptions-boxable.csv',header=None )
classes_data.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [25]:
classes = ['Fruit']

class_string = []

for i in classes:
  req_class = classes_data.loc[classes_data[1]==i]
  string = req_class.iloc[0][0]
  print(string)
  class_string.append(string)

print(class_string)

/m/02xwb
['/m/02xwb']


In [26]:
annotation_data = pd.read_csv('validation-annotations-bbox.csv',
                              usecols=['ImageID',
                              'LabelName',
                              'XMin',
                              'XMax',
                              'YMin',
                              'YMax'])
annotation_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
0,0001eeaf4aed83f9,/m/0cmf2,0.022464,0.964178,0.070656,0.800164
1,000595fe6fee6369,/m/02wbm,0.000000,1.000000,0.000233,1.000000
2,000595fe6fee6369,/m/02xwb,0.141030,0.180277,0.676262,0.732455
3,000595fe6fee6369,/m/02xwb,0.213781,0.253028,0.298764,0.354956
4,000595fe6fee6369,/m/02xwb,0.232926,0.288447,0.488954,0.545146


In [27]:
filtered_class_data = annotation_data.loc[annotation_data['LabelName'].isin(class_string)].copy()
filtered_class_data.head()

,ImageID,LabelName,XMin,XMax,YMin,YMax
2,000595fe6fee6369,/m/02xwb,0.141030,0.180277,0.676262,0.732455
3,000595fe6fee6369,/m/02xwb,0.213781,0.253028,0.298764,0.354956
4,000595fe6fee6369,/m/02xwb,0.232926,0.288447,0.488954,0.545146
5,000595fe6fee6369,/m/02xwb,0.245370,0.290361,0.661854,0.716605
6,000595fe6fee6369,/m/02xwb,0.245370,0.291319,0.548028,0.604220


In [28]:
filtered_class_data['classNumber']=''
filtered_class_data['center x']=''
filtered_class_data['center y']=''
filtered_class_data['width']=''
filtered_class_data['height']=''

In [29]:
for i in range(len(class_string)):
  filtered_class_data.loc[filtered_class_data['LabelName']==class_string[i] ,'classNumber'] = i

filtered_class_data['center x']=(filtered_class_data['XMax']+filtered_class_data['XMin'])/2
filtered_class_data['center y']=(filtered_class_data['YMax']+filtered_class_data['YMin'])/2

filtered_class_data['width'] = filtered_class_data['XMax']-filtered_class_data['XMin']
filtered_class_data['height'] = filtered_class_data['YMax']-filtered_class_data['YMin']

Yolo_values= filtered_class_data.loc[:,['ImageID','classNumber','center x','center y','width','height']].copy()
Yolo_values.head()



,ImageID,classNumber,center x,center y,width,height
2,000595fe6fee6369,0,0.160654,0.704358,0.039247,0.056193
3,000595fe6fee6369,0,0.233404,0.326860,0.039247,0.056192
4,000595fe6fee6369,0,0.260686,0.517050,0.055521,0.056192
5,000595fe6fee6369,0,0.267865,0.689230,0.044991,0.054751
6,000595fe6fee6369,0,0.268344,0.576124,0.045949,0.056192


In [30]:
image_path= '/content/drive/MyDrive/OID/Dataset/validation/Fruit'
os.chdir(image_path)

In [31]:
from numpy.lib.type_check import imag
for current_dir,dirs,files in os.walk('.'):
  for f in files:
    if f.endswith('.jpg'):
      image_title = f[:-4]

      Yolo_file =Yolo_values.loc[Yolo_values['ImageID']==image_title]
      df=Yolo_file.loc[:,['classNumber','center x','center y','width','height']].copy()
      save_path= image_path+'/'+image_title+'.txt'
      df.to_csv(save_path, header=False, index=False, sep=' ')